In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

Download the data

In [2]:
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [3]:
concrete_data.shape

(1030, 9)

In [4]:
#Let's check the dataset for any missing values.
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [5]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

# Split data into predictors and target
The target variable in this problem is the concrete sample strength. Therefore, our predictors will be all the other columns.

In [6]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

In [7]:
#Let's do a quick sanity check of the predictors and the target dataframes.
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [8]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

Finally, the last step is to normalize the data by substracting the mean and dividing by the standard deviation.

In [9]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [10]:
#Let's save the number of predictors to _n_cols_ since we will need this number when building our network.
n_cols = predictors_norm.shape[1] # number of predictors
n_cols

8

# Build a Neural Network


In [11]:
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='Adam', loss='mean_squared_error')
    return model

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state=4)

In [13]:
# build the model
model = regression_model()

# Train and Test the Network


In [14]:
# build the model
model.fit( X_test, y_test,epochs=50, verbose=2)

Epoch 1/50
10/10 - 0s - loss: 8160.4634
Epoch 2/50
10/10 - 0s - loss: 1034.0894
Epoch 3/50
10/10 - 0s - loss: 399.3633
Epoch 4/50
10/10 - 0s - loss: 498.2967
Epoch 5/50
10/10 - 0s - loss: 319.7950
Epoch 6/50
10/10 - 0s - loss: 284.8564
Epoch 7/50
10/10 - 0s - loss: 284.1665
Epoch 8/50
10/10 - 0s - loss: 271.0115
Epoch 9/50
10/10 - 0s - loss: 269.4097
Epoch 10/50
10/10 - 0s - loss: 265.3610
Epoch 11/50
10/10 - 0s - loss: 262.7644
Epoch 12/50
10/10 - 0s - loss: 259.2085
Epoch 13/50
10/10 - 0s - loss: 256.3792
Epoch 14/50
10/10 - 0s - loss: 251.9788
Epoch 15/50
10/10 - 0s - loss: 248.5820
Epoch 16/50
10/10 - 0s - loss: 245.0922
Epoch 17/50
10/10 - 0s - loss: 241.1706
Epoch 18/50
10/10 - 0s - loss: 238.4170
Epoch 19/50
10/10 - 0s - loss: 233.5254
Epoch 20/50
10/10 - 0s - loss: 230.9334
Epoch 21/50
10/10 - 0s - loss: 227.5243
Epoch 22/50
10/10 - 0s - loss: 225.5217
Epoch 23/50
10/10 - 0s - loss: 221.6321
Epoch 24/50
10/10 - 0s - loss: 219.8707
Epoch 25/50
10/10 - 0s - loss: 216.2176
Epoch 2

Model evaluation

In [15]:
loss = model.evaluate(X_test, y_test)
y_hat = model.predict(X_test)
loss

10/10 [==============================] - 0s 1ms/step - loss: 169.5988


169.5988006591797

Create a list of mean squared errors and  the standard deviation of the mean squared errors


In [16]:
total_mean_squared_errors = 50
mean_squared_errors = []
for i in range(0,total_mean_squared_errors):
    X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state=4)
    model.fit(X_train, y_train, epochs=50, verbose=0)
    MSE = model.evaluate(X_test, y_test, verbose=0)
    print("MSE "+str(i+1)+": "+str(MSE))
    y_hat = model.predict(X_test)
    errors = mean_squared_error(y_test, y_hat)
    mean_squared_errors.append(errors)
    
mean = np.mean(np.array(mean_squared_errors))
STD = np.std(mean_squared_errors)
print("Mean: ",str(mean), "Standard Deviation:",str(STD))


MSE 1: 95.84102630615234
MSE 2: 65.08763885498047
MSE 3: 54.68662643432617
MSE 4: 49.57232666015625
MSE 5: 46.0684700012207
MSE 6: 50.18879318237305
MSE 7: 50.390281677246094
MSE 8: 50.48544692993164
MSE 9: 49.804752349853516
MSE 10: 49.56337356567383
MSE 11: 50.20034408569336
MSE 12: 48.627342224121094
MSE 13: 41.930477142333984
MSE 14: 40.57073974609375
MSE 15: 42.680965423583984
MSE 16: 45.427555084228516
MSE 17: 39.663719177246094
MSE 18: 42.70329666137695
MSE 19: 40.62495422363281
MSE 20: 39.22114181518555
MSE 21: 40.45365905761719
MSE 22: 41.44636154174805
MSE 23: 38.41522216796875
MSE 24: 37.63630294799805
MSE 25: 38.22797393798828
MSE 26: 39.02134323120117
MSE 27: 39.47100830078125
MSE 28: 42.819793701171875
MSE 29: 47.98396682739258
MSE 30: 37.1829948425293
MSE 31: 37.341285705566406
MSE 32: 38.19464874267578
MSE 33: 40.41322708129883
MSE 34: 37.25727844238281
MSE 35: 37.552207946777344
MSE 36: 44.03314971923828
MSE 37: 37.379512786865234
MSE 38: 39.75529861450195
MSE 39: 52.4

In [17]:
experiment_name='Experiment'
MMSE='MMSE'
Standard_Dev='STD'
report_header = [experiment_name, MMSE,Standard_Dev]
def get_report(name_of_experiment, mean, STD):
    
    values = [[experiment_name, mean, STD]]

    return pd.DataFrame(columns=report_header, data=values)

experiment_name='Experiment4'

get_report(experiment_name, mean, STD)

,Experiment,MMSE,STD
0,Experiment4,43.679129,9.492925


we see that when when we increase the hidden layers the mean of the mean squared errors is a lot better than in part B